In [12]:
import os
import numpy as np

In [13]:
def LoadArr(filename, type):
    """
    Load the data array from the file "filename" into a PL numpy array of type "type"
    """
    arr = []
    with open(filename) as f:
        line = f.readline()
        num = 0
        while line:
            arr.append(type(line))
            num += 1
            line = f.readline()
    return arr, num

In [14]:
def GetData(filename):
    """
    Print the data array "arr" with "num" elements
    """
    fp = open(filename, "r")
    result = ""
    cnt = 0
    line = fp.readline()
    while line:
        cnt += 1
        result = result + str(line[:-1])
        result = result + ', '
        line = fp.readline()
    fp.close()
    result = '[' + str(cnt) + '] = {' + result[:-2] + '};'
    return result

In [15]:
file_prefix = "C:/Users/Lenovo/source/repos/ConsoleApplication1/ConsoleApplication1/parameters_quantized_flatten/parameters_quantized_flatten/"
file_list = []
file_list = os.listdir(file_prefix)

# split with '.'
for i in range(len(file_list)):
    file_list[i] = file_list[i].split('.')

# remove folders
for file_name in file_list:
    if file_name[-1] != 'dat' or len(file_name) <= 2:
        file_list.remove(file_name)
for file_name in file_list:
    if file_name[-1] != 'dat' or len(file_name) <= 2:
        file_list.remove(file_name)


In [16]:
# get scale
scale_list = []
for file_name in file_list:
    if file_name[-2] == 'scale':
        scale_list.append(file_name)

# get running_mean
running_mean_list = []
for file_name in file_list:
    if file_name[-2] == 'running_mean':
        running_mean_list.append(file_name)

# get running_var
running_var_list = []
for file_name in file_list:
    if file_name[-2] == 'running_var':
        running_var_list.append(file_name)

# get bias
bias_list = []
for file_name in file_list:
    if file_name[-2] == 'bias':
        bias_list.append(file_name)

# get gamma
gamma_list = []
for file_name in file_list:
    if file_name[-2] == 'weight' and (file_name[-3] == '1' or file_name[-3] == '4') and file_name[0] != 'fc':
        gamma_list.append(file_name)



In [19]:
# create a .h file
with open("../inner_data.cpp", "w") as f:
    fh = open("../inner_data.h", "w")
    print("#ifndef INNER_DATA_H", file=fh)
    print("#define INNER_DATA_H", file=fh)
    print('#include "r2plus1d.h"', file=fh)
    print("", file=fh)

    print('#include "inner_data.h"', file=f)
    print("", file=f)
    # print scale
    print("// scales", file=f)
    for scale_name in scale_list:
        data_type = 'ftype'
        arr = GetData(file_prefix + '.'.join(scale_name))
        if scale_name[0] == 'fc':
            var_name = 'Kernel_linear_scale'
        elif scale_name[2] == 'downsample':
            var_name = 'Kernel_' + scale_name[0].replace('layer', 'seq') + '_' + scale_name[1] + '_' + scale_name[2] + '_' + scale_name[3] + '_scale'
        elif scale_name[0] == 'stem':
            var_name = 'Kernel_' + scale_name[0] + '_' + scale_name[1] + '_scale'
        else:
            var_name = 'Kernel_' + scale_name[0].replace('layer', 'seq') + '_' + scale_name[1] + '_' + scale_name[2] + '_' + scale_name[3] + '_' + scale_name[4] + '_scale'
        print(data_type + ' '+ var_name  + arr, file=f)
        print('extern ' + data_type + ' '+ var_name  + arr.split(' ')[0] + ';', file=fh)
    
    # print running_mean
    print("", file=f)
    print("// running_means", file=f)
    for running_mean_name in running_mean_list:
        data_type = 'ftype'
        arr = GetData(file_prefix + '.'.join(running_mean_name))
        if running_mean_name[0] == 'stem':
            var_name = 'Mu_' + running_mean_name[0] + '_' + running_mean_name[1]
        else:
            var_name = 'Mu_' + running_mean_name[0].replace('layer', 'seq') + '_' + running_mean_name[1] + '_' + running_mean_name[2] + '_' + '_'.join(running_mean_name[3:-2])
        print(data_type + ' '+ var_name  + arr, file=f)
        print('extern ' + data_type + ' '+ var_name  + arr.split(' ')[0] + ';', file=fh)

    # print running_var
    print("", file=f)
    print("// running_vars", file=f)
    for running_var_name in running_var_list:
        data_type = 'ftype'
        arr = GetData(file_prefix + '.'.join(running_var_name))
        if running_var_name[0] == 'stem':
            var_name = 'Var_' + running_var_name[0] + '_' + running_var_name[1]
        else:
            var_name = 'Var_' + running_var_name[0].replace('layer', 'seq') + '_' + running_var_name[1] + '_' + running_var_name[2] + '_' + '_'.join(running_var_name[3:-2])
        print(data_type + ' '+ var_name  + arr, file=f)
        print('extern ' + data_type + ' '+ var_name  + arr.split(' ')[0] + ';', file=fh)
    
    # print bias
    print("", file=f)
    print("// biases", file=f)
    for bias_name in bias_list:
        data_type = 'ftype'
        arr = GetData(file_prefix + '.'.join(bias_name))
        if bias_name[0] == 'stem':
            var_name = 'Bias_' + bias_name[0] + '_' + bias_name[1]
        else:
            var_name = 'Bias_' + bias_name[0].replace('layer', 'seq') + '_' + bias_name[1] + '_' + bias_name[2] + '_' + '_'.join(bias_name[3:-2])
        print(data_type + ' '+ var_name  + arr, file=f)
        print('extern ' + data_type + ' '+ var_name  + arr.split(' ')[0] + ';', file=fh)

    # print gamma
    print("", file=f)
    print("// gammas", file=f)
    for bias_name in gamma_list:
        data_type = 'ftype'
        arr = GetData(file_prefix + '.'.join(bias_name))
        if bias_name[0] == 'stem':
            var_name = 'Gamma_' + bias_name[0] + '_' + bias_name[1]
        else:
            var_name = 'Gamma_' + bias_name[0].replace('layer', 'seq') + '_' + bias_name[1] + '_' + bias_name[2] + '_' + '_'.join(bias_name[3:-2])
        print(data_type + ' '+ var_name  + arr, file=f)
        print('extern ' + data_type + ' '+ var_name  + arr.split(' ')[0] + ';', file=fh)
    
    print("#endif", file=fh)
    fh.close()
